In [2]:
!pip install --upgrade accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 57.4 MB/s  0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 42.9 MB/s  0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.9.0
    Uninstalling accelerate-1.9.0:
      Successfully uninstalled accelerate-1.9.0
  Attempting uninstall: transformers0m╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [accelerate]
    Found existing installation: transformers 4.55.0.dev0237m━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [accelerate]
    Uninstalling transformers-4.55.0.dev0:━━━━━╸━━━━━━━━━━━━━ 2/3 [transformers]
      Successfully uninstalled transformers-4.55.0.dev049;38;114m╸━━━━━━━━━━━━━ 2/3 [transformers]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [transformers]━━━━━ 2/3 [transformers]
ERROR: pip's dependency resolver does not curre

In [1]:
import sys
print(sys.executable)

/Users/kangheecho/PycharmProjects/forecasting/venv/bin/python


In [19]:
import pandas as pd
import json

In [15]:
# Load your cleaned data
df = pd.read_csv("trump_data_clean.csv", converters={"transcript": json.loads})

# Flatten it into a single long string file
with open("train.txt", "w", encoding="utf-8") as f:
    for idx, row in df.iterrows():
        if type(row['event_name']) is not str:
            continue
        # Write the Mode tag
        f.write(f"<|startoftext|> [MODE: {row['event_name'].split(':')[0]}] [TITLE: {row['event_name'].split(':')[1]}]\n")

        for block in row['transcript']:
            role = "[TRUMP]" if "donald trump" in block['speaker'].lower() else "[SPEAKER]"
            f.write(f"{role}: {block['remark']}\n")

        f.write("<|endoftext|>\n\n")

print("Created train.txt")

Created train.txt


In [2]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

# 1. Configuration
MODEL_NAME = "gpt2"
TRAIN_FILE = "train.txt"
OUTPUT_DIR = "./trump_model_v1"

# 2. Load Tokenizer & Model
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

# 3. Prepare Dataset
print("Loading dataset (this may take a minute)...")
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=TRAIN_FILE,
    block_size=1024
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# 4. Training Settings
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    num_train_epochs=1,

    per_device_train_batch_size=4,   # Keep small (4-8) to avoid "Out of Memory"
    gradient_accumulation_steps=8,   # Accumulate gradients to simulate a batch size of 32 (4 * 8)

    dataloader_num_workers=0,        # CRITICAL: Multi-processing data loaders often crash on Mac. Set to 0.

    # --- Saving & Logging ---
    save_steps=500,
    save_total_limit=2,
    logging_steps=50,                # Log more frequently to ensure it's not stuck
    prediction_loss_only=True,
)

# 5. Train
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

print("Starting training...")
trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Model saved to {OUTPUT_DIR}")

Loading dataset (this may take a minute)...


/Users/kangheecho/PycharmProjects/forecasting/venv/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Starting training...


/Users/kangheecho/PycharmProjects/forecasting/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,2.799700
100,2.717500
150,2.693600
200,2.676300
250,2.657100


Model saved to ./trump_model_v1
